## 1-1. YOLO 기반 실시간 낙상 모니터링(Hugging face)

In [ ]:
# yolo_fall_only_sticky_visible_only.py
from ultralytics import YOLO
import cv2
import time
import torch
import logging
import os
from collections import defaultdict

logging.getLogger("ultralytics").setLevel(logging.ERROR)

# ===== 0) 가중치 경로 =====
LOCAL_FALL_WEIGHTS = r"C:\models\fall.pt"  # 로컬 경로가 있으면 설정, 없으면 None
HF_REPO = "onebeans/YOLO_safetyfactory"
HF_FILENAME = "fall.pt"

def resolve_fall_weights():
    if LOCAL_FALL_WEIGHTS and os.path.isfile(LOCAL_FALL_WEIGHTS):
        return LOCAL_FALL_WEIGHTS
    try:
        from huggingface_hub import hf_hub_download
        print("[INFO] 로컬에 가중치가 없어 허깅페이스에서 다운로드합니다…")
        return hf_hub_download(repo_id=HF_REPO, filename=HF_FILENAME)
    except Exception as e:
        raise SystemExit(
            "[ERROR] fall.pt 경로를 찾을 수 없고, 허깅페이스에서도 받지 못했습니다.\n"
            f" - LOCAL_FALL_WEIGHTS를 실제 경로로 바꾸거나,\n"
            f" - pip install huggingface_hub 후 다시 실행하세요.\n원인: {e}"
        )

# ===== 1) 디바이스/모델 =====
DEVICE = 0 if torch.cuda.is_available() else "cpu"
HALF = (DEVICE != "cpu")

FALL_WEIGHTS = resolve_fall_weights()
model_fall = YOLO(FALL_WEIGHTS)
try:
    model_fall.fuse()
except Exception:
    pass

# ===== 2) 파라미터 =====
IMGSZ = 640
CONF = 0.5
IOU  = 0.45

# 최근 fall로 찍히면 그 상태를 유지할 프레임 수 (예: 60 = 2초@30fps)
STICKY_FRAMES = 60

# 추적기
TRACKER_CFG = "bytetrack.yaml"

# ===== 3) 라벨 그리기(안전 배경 포함) =====
def draw_label_with_bg(
    frame, text, x1, y1, color=(0, 255, 0),
    font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.6, thickness=2
):
    h, w = frame.shape[:2]
    (tw, th), baseline = cv2.getTextSize(text, font, font_scale, thickness)
    tx = max(0, min(x1, w - tw - 1))
    ty = y1 - 5
    if ty - th - baseline < 0:
        ty = min(h - 1, y1 + th + 6)

    bg_x1 = tx
    bg_y1 = ty - th - baseline - 2
    bg_x2 = min(w - 1, tx + tw + 2)
    bg_y2 = min(h - 1, ty + 2)

    cv2.rectangle(frame, (bg_x1, bg_y1), (bg_x2, bg_y2), color, -1)
    cv2.putText(frame, text, (tx, ty), font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)

def draw_box(frame, xyxy, color=(0,255,0), thickness=2):
    x1, y1, x2, y2 = map(int, xyxy)
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness, lineType=cv2.LINE_AA)

# ===== 4) fall 클래스 id 추출 =====
def get_fall_class_ids(names_dict):
    fall_ids = {cid for cid, cname in names_dict.items() if "fall" in str(cname).lower()}
    if not fall_ids and len(names_dict) == 1:
        fall_ids = {0}
    return fall_ids

# ===== 5) 메인 =====
def run_fall_on_video_sticky_visible_only(
    video_path: str,
    save_path: str | None = None,
    show: bool = True,
    imgsz: int = IMGSZ,
    conf: float = CONF,
    iou: float = IOU,
    device=DEVICE,
    half: bool = HALF,
):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"동영상을 열 수 없습니다: {video_path}")

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

    out = None
    if save_path:
        os.makedirs(os.path.dirname(save_path) or ".", exist_ok=True)
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(save_path, fourcc, fps, (w, h))

    # 추적 + persist 켜서 ID 유지
    common_args = dict(
        imgsz=imgsz, conf=conf, iou=iou,
        device=device, half=half,
        verbose=False, stream=False, persist=True,
        tracker=TRACKER_CFG
    )

    names    = model_fall.model.names
    fall_ids = get_fall_class_ids(names)

    # 스티키 만료 프레임 기록
    sticky_until = defaultdict(int)  # tid -> expire_frame

    frame_i = 0
    t0 = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model_fall.track(frame, **common_args)

        if results:
            for r in results:
                if not hasattr(r, "boxes") or r.boxes is None or len(r.boxes) == 0:
                    continue

                boxes = r.boxes
                ids   = getattr(boxes, "id", None)

                for i, box in enumerate(boxes):
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    conf = float(box.conf.item())
                    cls  = int(box.cls.item())

                    # 추적 ID
                    tid = None
                    if ids is not None and len(ids) > i and ids[i] is not None:
                        tid = int(ids[i].item())
                    if tid is None:
                        # 추적 ID가 없으면 해당 박스는 스티키 관리 없이 그냥 그리기
                        is_fall_now = (cls in fall_ids) or ("fall" in str(r.names.get(cls, "")).lower())
                        color = (0,0,255) if is_fall_now else (0,255,0)
                        label = "fall" if is_fall_now else r.names.get(cls, str(cls))
                        draw_box(frame, (x1,y1,x2,y2), color=color)
                        draw_label_with_bg(frame, f"{label} {conf:.2f}", int(x1), int(y1), color=color)
                        continue

                    # 현재 프레임에서 fall 감지면 스티키 연장
                    is_fall_now = (cls in fall_ids) or ("fall" in str(r.names.get(cls, "")).lower())
                    if is_fall_now:
                        sticky_until[tid] = max(sticky_until[tid], frame_i + STICKY_FRAMES)

                    # 스티키가 살아있으면 fall로 표기(빨강), 아니면 원래 클래스(초록)
                    if sticky_until[tid] > frame_i:
                        draw_box(frame, (x1,y1,x2,y2), color=(0,0,255))
                        draw_label_with_bg(frame, f"fall {conf:.2f}", int(x1), int(y1), color=(0,0,255))
                    else:
                        label = r.names.get(cls, str(cls))
                        draw_box(frame, (x1,y1,x2,y2), color=(0,255,0))
                        draw_label_with_bg(frame, f"{label} {conf:.2f}", int(x1), int(y1), color=(0,255,0))

        # 저장/표시
        if out:
            out.write(frame)
        if show:
            cv2.imshow("YOLO Fall-only (sticky / visible only)", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        frame_i += 1

    cap.release()
    if out:
        out.release()
        print(f"[INFO] 저장 완료: {save_path}")
    if show:
        cv2.destroyAllWindows()

    dt = time.time() - t0
    print(f"[INFO] 총 프레임: {frame_i}, 총 시간: {dt:.2f}s, 평균 FPS: {frame_i / max(dt, 1e-6):.2f}")

# ===== 실행 예시 =====
if __name__ == "__main__":
    video_path = r"Worker Falls Wearing Fall Protection.mp4"
    # video_path = r"Falling from height.mp4"
    video_path = r"C:\Users\main\Desktop\강현\산업안전\낙상\cctv_fall\multi_people.mp4"
    save_path  = r"C:\workspace\industrial_safety\fall\test_result_video\4_1.mp4"
    run_fall_on_video_sticky_visible_only(video_path, save_path=save_path, show=True)


## 1-2. 사람 탐지 후 YOLO 기반 실시간 낙상 모니터링(Hugging face)

In [ ]:
# yolo_person_then_fall_sticky_visible_only.py
from ultralytics import YOLO
import cv2, time, torch, logging, os
from collections import defaultdict

logging.getLogger("ultralytics").setLevel(logging.ERROR)

# ===== 0) 가중치 경로 =====
# - 사람 검출: COCO person
LOCAL_PERSON_WEIGHTS = r"C:\models\yolo11s.pt"     # 없으면 모델 이름만으로도 동작할 수 있음
# - 낙상 모델: 허깅페이스에서 받았던 fall.pt (또는 로컬 경로)
LOCAL_FALL_WEIGHTS   = r"C:\models\fall.pt"        # 없으면 아래 resolve_fall_weights()가 HF에서 시도

HF_REPO = "onebeans/YOLO_safetyfactory"
HF_FILENAME = "fall.pt"

def resolve_fall_weights():
    if LOCAL_FALL_WEIGHTS and os.path.isfile(LOCAL_FALL_WEIGHTS):
        return LOCAL_FALL_WEIGHTS
    try:
        from huggingface_hub import hf_hub_download
        print("[INFO] 로컬에 fall.pt 없음 → 허깅페이스에서 다운로드…")
        return hf_hub_download(repo_id=HF_REPO, filename=HF_FILENAME)
    except Exception as e:
        raise SystemExit(
            "[ERROR] fall.pt을 찾지 못했고, 허깅페이스 다운로드도 실패.\n"
            " - LOCAL_FALL_WEIGHTS 경로를 실제 파일로 바꾸거나,\n"
            " - pip install huggingface_hub 후 재실행하세요.\n"
            f"원인: {e}"
        )

def resolve_person_weights():
    if LOCAL_PERSON_WEIGHTS and os.path.isfile(LOCAL_PERSON_WEIGHTS):
        return LOCAL_PERSON_WEIGHTS
    return "yolo11s.pt"  # 이름만 넘겨도 다운로드/캐시되는 경우가 많음

# ===== 1) 디바이스/모델 =====
DEVICE = 0 if torch.cuda.is_available() else "cpu"
HALF = (DEVICE != "cpu")

FALL_WEIGHTS   = resolve_fall_weights()
PERSON_WEIGHTS = resolve_person_weights()

model_person = YOLO(PERSON_WEIGHTS)  # 사람 추적 전용
model_fall   = YOLO(FALL_WEIGHTS)    # 낙상 분류/검출

for m in (model_person, model_fall):
    try:
        m.fuse()
    except Exception:
        pass

# ===== 2) 파라미터 =====
IMGSZ_PERSON = 640
CONF_PERSON  = 0.35     # 사람은 넓게 잡기    0.35
IOU_PERSON   = 0.45

IMGSZ_FALL = 512        # 크롭에 돌릴 입력 크기(낙상)
CONF_FALL  = 0.55       # 낙상 임계값 약간 상향    0.55
IOU_FALL   = 0.45

FALL_EVERY = 1          # 낙상 추론 주기(1=매 프레임, 2=격프레임)
CROP_PAD   = 0.10       # 사람 박스 크롭 시 상하좌우 여유 비율   0.10

# 스티키(보이는 프레임에서만 유지)
STICKY_FRAMES = 60      # 60프레임=약 2초@30fps
TRACKER_CFG   = "bytetrack.yaml"

# 너무 작은/이상한 박스 제거
MIN_AREA = 24 * 24
ASPECT_MIN, ASPECT_MAX = 0.25, 1.55  # 세로형 위주 허용

# ===== 3) 도우미 =====
def clamp(val, lo, hi): return max(lo, min(hi, val))

def expand_and_clip(box, pad_ratio, W, H):
    """ box(x1,y1,x2,y2)를 pad_ratio만큼 확장 후 영상 경계로 클램프 """
    x1, y1, x2, y2 = box
    w, h = x2 - x1, y2 - y1
    px, py = w * pad_ratio, h * pad_ratio
    nx1 = clamp(int(x1 - px), 0, W - 1)
    ny1 = clamp(int(y1 - py), 0, H - 1)
    nx2 = clamp(int(x2 + px), 0, W - 1)
    ny2 = clamp(int(y2 + py), 0, H - 1)
    return nx1, ny1, nx2, ny2

def draw_box(frame, xyxy, color=(0,255,0), thickness=2):
    x1, y1, x2, y2 = map(int, xyxy)
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness, lineType=cv2.LINE_AA)

def draw_label_with_bg(frame, text, x1, y1, color=(0,255,0),
                       font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.6, thickness=2):
    h, w = frame.shape[:2]
    (tw, th), base = cv2.getTextSize(text, font, font_scale, thickness)
    tx = max(0, min(int(x1), w - tw - 1))
    ty = int(y1) - 5
    if ty - th - base < 0:
        ty = min(h - 1, int(y1) + th + 6)
    bg_x1, bg_y1 = tx, ty - th - base - 2
    bg_x2, bg_y2 = min(w - 1, tx + tw + 2), min(h - 1, ty + 2)
    cv2.rectangle(frame, (bg_x1, bg_y1), (bg_x2, bg_y2), color, -1)
    cv2.putText(frame, text, (tx, ty), font, font_scale, (255,255,255), thickness, lineType=cv2.LINE_AA)

def get_fall_class_ids(names_dict):
    fall_ids = {cid for cid, cname in names_dict.items() if "fall" in str(cname).lower()}
    if not fall_ids and len(names_dict) == 1:
        fall_ids = {0}
    return fall_ids

# ===== 4) 메인 =====
def run(video_path, save_path=None, show=True):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"동영상을 열 수 없습니다: {video_path}")

    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

    out = None
    if save_path:
        os.makedirs(os.path.dirname(save_path) or ".", exist_ok=True)
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(save_path, fourcc, fps, (W, H))

    # 사람 추적 설정(persist=True로 ID 유지)
    person_args = dict(
        imgsz=IMGSZ_PERSON, conf=CONF_PERSON, iou=IOU_PERSON,
        device=DEVICE, half=HALF, verbose=False, stream=False,
        persist=True, tracker=TRACKER_CFG
    )
    # 낙상 추론(사람 크롭에만 실행)
    fall_args = dict(
        imgsz=IMGSZ_FALL, conf=CONF_FALL, iou=IOU_FALL,
        device=DEVICE, half=HALF, verbose=False, stream=False
    )

    # 낙상 클래스 id
    fall_names = model_fall.model.names
    fall_ids = get_fall_class_ids(fall_names)

    # 스티키 만료 (사람 트랙 ID 기준)
    sticky_until = defaultdict(int)

    frame_i = 0
    t0 = time.time()

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        # 1) 사람 추적 (매 프레임)
        presults = model_person.track(frame, **person_args)

        # 2) 사람별 크롭에만 낙상 모델 적용
        if presults:
            for pr in presults:
                if not hasattr(pr, "boxes") or pr.boxes is None or len(pr.boxes) == 0:
                    continue

                boxes = pr.boxes
                ids   = getattr(boxes, "id", None)

                for i, pbox in enumerate(boxes):
                    cls = int(pbox.cls.item())
                    if cls != 0:  # COCO person=0
                        continue

                    # 사람 박스/ID
                    px1, py1, px2, py2 = pbox.xyxy[0].tolist()
                    tid = None
                    if ids is not None and len(ids) > i and ids[i] is not None:
                        tid = int(ids[i].item())

                    # 간단한 형태/크기 필터
                    w_box, h_box = max(1, px2 - px1), max(1, py2 - py1)
                    area = w_box * h_box
                    aspect = h_box / w_box
                    if area < MIN_AREA or not (ASPECT_MIN <= aspect <= ASPECT_MAX):
                        continue

                    # 크롭 준비(패딩 포함)
                    cx1, cy1, cx2, cy2 = expand_and_clip((px1, py1, px2, py2), CROP_PAD, W, H)
                    crop = frame[cy1:cy2, cx1:cx2]
                    if crop.size <= 0:
                        continue

                    # 낙상 추론은 FALL_EVERY 프레임마다만(속도 최적화)
                    is_fall_now = False
                    conf_out = 0.0
                    if frame_i % FALL_EVERY == 0:
                        fres = model_fall.predict(crop, **fall_args)
                        # 크롭 내 낙상 결과가 있으면 true
                        if fres:
                            for fr in fres:
                                if not hasattr(fr, "boxes") or fr.boxes is None:
                                    continue
                                for fbox in fr.boxes:
                                    fcls = int(fbox.cls.item())
                                    fconf = float(fbox.conf.item())
                                    if fcls in fall_ids:
                                        is_fall_now = True
                                        conf_out = max(conf_out, fconf)

                    # 스티키 연장(이 사람 트랙이 fall로 평가되면)
                    if tid is not None and is_fall_now:
                        sticky_until[tid] = max(sticky_until[tid], frame_i + STICKY_FRAMES)

                    # 시각화: 사람 박스만 그림 (보이는 프레임에서만)
                    if tid is not None and sticky_until[tid] > frame_i:
                        color = (0, 0, 255)
                        label = f"fall {conf_out:.2f}" if conf_out > 0 else "fall"
                    else:
                        color = (0, 255, 0)
                        label = "person"

                    draw_box(frame, (px1, py1, px2, py2), color=color)
                    draw_label_with_bg(frame, label, int(px1), int(py1), color=color)

        # 3) 저장/표시
        if out:
            out.write(frame)
        if show:
            cv2.imshow("Person → Fall (sticky, visible-only)", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        frame_i += 1

    cap.release()
    if out:
        out.release()
        print(f"[INFO] 저장 완료: {save_path}")
    if show:
        cv2.destroyAllWindows()

    dt = time.time() - t0
    print(f"[INFO] 총 프레임: {frame_i}, 총 시간: {dt:.2f}s, 평균 FPS: {frame_i/max(dt,1e-6):.2f}")

# ===== 실행 예시 =====
if __name__ == "__main__":
    video_path = r"C:\Users\main\Desktop\강현\산업안전\낙상\cctv_fall\multi_people.mp4"
    # video_path = r"Falling from height.mp4"
    save_path  = r"C:\workspace\industrial_safety\fall\test_result_video\3-3.mp4"
    run(video_path, save_path=save_path, show=True)


## 2. 중심축 각도 기반 탐지

In [ ]:
from dataclasses import dataclass
from collections import deque, defaultdict
from typing import Dict, Optional, Tuple, List
import math
import time
import argparse
import os
import csv
import cv2

# --------- 유틸 ---------

# ---- 시각화 임계값(수직 기준 기울기) ----
TILT_YELLOW_DEG = 15.0  # 15° 이상이면 노란색
TILT_RED_DEG    = 30.0  # 30° 이상이면 빨간색

def angle_to_ground_from_axis(ax: float, ay: float, vert=(0.0, -1.0)) -> float:
    """
    현재 코드의 angle_deg는 (몸축 vs '수직') 각도(0=수직, 90=수평)를 반환한다.
    지면(수평)과 몸축 사이 각도 θ는 90 - angle_vertical 로 계산한다.
    반환 범위: [0, 90]
    """
    ang_v = angle_deg(ax, ay, vert[0], vert[1])  # 몸축 vs 수직(up) 각도
    theta = max(0.0, min(90.0, 90.0 - ang_v))   # 몸축 vs 수평(지면) 각도
    return ang_v

def draw_ground_angle_guides(frame, origin_xy: Tuple[int, int], theta_deg: float, length: int = 90):
    """
    origin_xy: 가이드 시작점 (보통 '발(lower)' 좌표)
    theta_deg: 몸축-지면 각도(0~90)
    length   : 가이드 선 길이
    그려주는 것:
      - 0° 기준 수평선(얕은 회색 점선)
      - 15°(노란색), 30°(빨간색) 가이드 라인 (좌우 대칭)
      - 실제 θ 라벨 텍스트
    """
    import cv2, math

    x0, y0 = origin_xy

    # 수평 기준선(얕은 회색 점선)
    dash = 6
    for i in range(0, length, dash*2):
        cv2.line(frame, (x0 + i, y0), (x0 + i + dash, y0), (180,180,180), 1, cv2.LINE_AA)
        cv2.line(frame, (x0 - i, y0), (x0 - i - dash, y0), (180,180,180), 1, cv2.LINE_AA)

    # 각도별 가이드 (지면 기준으로 위쪽으로 그린다)
    for deg, color in [(90 - TILT_YELLOW_DEG, (0,255,255)), (90 - TILT_RED_DEG, (0,0,255))]:  # 15°=노란색, 30°=빨간색 (BGR)
        rad = math.radians(deg)
        dx = int(math.cos(rad) * length)
        dy = int(math.sin(rad) * length)
        cv2.line(frame, (x0, y0), (x0 + dx, y0 - dy), color, 2, cv2.LINE_AA)  # 오른쪽 위
        cv2.line(frame, (x0, y0), (x0 - dx, y0 - dy), color, 2, cv2.LINE_AA)  # 왼쪽 위

    # 실제 θ 라벨
    label = f"θ={theta_deg:.1f}°"
    cv2.putText(frame, label, (x0 + 8, y0 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(frame, label, (x0 + 8, y0 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0,0,0),   1, cv2.LINE_AA)

def ema(prev: float, new: float, alpha: float) -> float:
    return alpha*new + (1-alpha)*prev

def angle_deg(vx: float, vy: float, ux: float, uy: float) -> float:
    # angle between v=(vx,vy) and u=(ux,uy)
    dot = vx*ux + vy*uy
    nv = math.hypot(vx, vy)
    nu = math.hypot(ux, uy)
    if nv == 0 or nu == 0:
        return 0.0
    cosv = max(-1.0, min(1.0, dot/(nv*nu)))
    return math.degrees(math.acos(cosv))

def median(lst: List[float]) -> float:
    s = sorted(lst)
    n = len(s)
    if n == 0: return 0.0
    m = n//2
    return (s[m] if n%2 else 0.5*(s[m-1]+s[m]))

@dataclass
class PosePoints:
    # 필요한 점만 사용: 상부(머리/목/어깨 중앙), 하부(발목 중앙)
    upper: Tuple[float, float]   # (x,y)
    lower: Tuple[float, float]   # (x,y)
    head:  Tuple[float, float]   # 머리/목 대용
    ankles: Tuple[float, float]  # 양발 평균

@dataclass
class DetectionIn:
    track_id: int
    t: float                # seconds
    pose: Optional[PosePoints] = None
    bbox: Optional[Tuple[float,float,float,float]] = None  # (x,y,w,h) optional fallback
    frame_w: Optional[int] = None
    frame_h: Optional[int] = None

@dataclass
class Event:
    track_id: int
    t: float
    kind: str       # "fall" | "fall_from_height" | "fall_warning"
    score: float
    angle: float
    vy_peak: float

class AngleFallDetector:
    def __init__(
        self,
        fps: float = 30.0,
        stand_max_angle: float = 25.0,
        lie_angle: float = 15.0,
        min_lie_frames: int = 1,
        fall_window_sec: float = 1.2,
        sticky_sec: float = 2.0,
        vy_fall_thresh: float = 400.0,
        vy_freefall_high: float = 900.0,
        inversion_min_frames: int = 3,
        horizon_vertical: Tuple[float,float] = (0.0, -1.0), # 보정된 "수직" 방향 (위가 음의 y)
        angle_ema_alpha: float = 0.4,
        vy_med_k: int = 5
    ):
        self.fps = fps
        self.STAND_MAX_ANGLE  = stand_max_angle
        self.LIE_ANGLE        = lie_angle
        self.MIN_LIE_FRAMES   = min_lie_frames
        self.FALL_WINDOW_SEC  = fall_window_sec
        self.STICKY_SEC       = sticky_sec
        self.VY_FALL_THRESH   = vy_fall_thresh
        self.VY_FREEFALL_HIGH = vy_freefall_high
        self.INV_MIN_FRAMES   = inversion_min_frames
        self.VERT = horizon_vertical
        self.ALPHA_ANGLE      = angle_ema_alpha
        self.VY_MED_K         = vy_med_k

        # 보수화 파라미터(색상기준 이벤트용)
        self.RED_MIN_FRAMES = 2     # 빨강 유지 최소 프레임
        self.YELLOW_MIN_FRAMES = 2  # 노랑 유지 최소 프레임
        self.YELLOW_MIN_VY = 0.0    # 필요하면 50~100 정도로 올려도 됨

        # per track state
        self.state = defaultdict(lambda: {
            "angle_ema": None,
            "last_t": None,
            "cy": None,
            "vy_hist": deque(maxlen=60),
            "angle_hist": deque(maxlen=90),
            "inversion_run": 0,
            "lie_run": 0,
            "is_lie": False,
            "last_event_t": -1e9,
            # 색상기준 유지 카운터
            "red_run": 0,
            "yellow_run": 0,
        })

    def _body_axis(self, det: DetectionIn) -> Optional[Tuple[float,float]]:
        if det.pose:
            (ux, uy) = det.pose.upper
            (lx, ly) = det.pose.lower
            return (ux - lx, uy - ly)  # 하부->상부(발→머리) 벡터
        elif det.bbox:
            # bbox만 있을 때는 세로축을 몸축 근사
            (_, _, _, h) = det.bbox
            return (0.0, -max(h,1.0))
        return None

    def _center_y(self, det: DetectionIn) -> float:
        if det.pose:
            (ux, uy) = det.pose.upper
            (lx, ly) = det.pose.lower
            return 0.5*(uy+ly)
        elif det.bbox:
            (x,y,w,h) = det.bbox
            return y + 0.5*h
        return 0.0

    def _is_inverted(self, det: DetectionIn) -> bool:
        if det.pose:
            head_y = det.pose.head[1]
            ank_y  = det.pose.ankles[1]
            # 화면 좌표: 아래로 갈수록 y 증가 → "발이 머리보다 위" = ankles_y < head_y
            return ank_y < head_y
        return False

    def _ground_near(self, det: DetectionIn) -> bool:
        if det.frame_h is None:
            return False
        cy = self._center_y(det)
        return cy > 0.9*det.frame_h  # 바닥 근접(해상도마다 튜닝)

    def step(self, det: DetectionIn) -> List[Event]:
        st = self.state[det.track_id]
        out: List[Event] = []

        # 1) 각도 계산
        axis = self._body_axis(det)
        if axis is None:
            return out
        angle = angle_deg(axis[0], axis[1], self.VERT[0], self.VERT[1])

        # EMA 평활화
        if st["angle_ema"] is None:
            st["angle_ema"] = angle
        else:
            st["angle_ema"] = ema(st["angle_ema"], angle, self.ALPHA_ANGLE)
        st["angle_hist"].append((det.t, st["angle_ema"]))

        # 2) 수직 속도 vy
        cy = self._center_y(det)
        if st["last_t"] is not None and st["cy"] is not None:
            dt = max(1e-3, (det.t - st["last_t"]))
            vy = (cy - st["cy"]) / dt  # +: 아래로 이동
            st["vy_hist"].append((det.t, vy))
        st["last_t"] = det.t
        st["cy"] = cy

        # 3) inversion / lie 지속 카운트
        inv = self._is_inverted(det)
        if inv: st["inversion_run"] += 1
        else:   st["inversion_run"]  = 0

        if st["angle_ema"] >= self.LIE_ANGLE:
            st["lie_run"] += 1
        else:
            st["lie_run"]  = 0

        # 상태 플래그
        was_lie = st["is_lie"]
        is_lie = (st["lie_run"] >= self.MIN_LIE_FRAMES)
        st["is_lie"] = is_lie

        # 4) 최근 창에서 피크 vy
        win_t0 = det.t - self.FALL_WINDOW_SEC
        vy_vals = [vy for (t,vy) in st["vy_hist"] if t >= win_t0]
        vy_peak = max(vy_vals) if vy_vals else 0.0
        vy_med  = median(vy_vals) if vy_vals else 0.0

        # 4.5) 색상기준 유지 카운트
        if st["angle_ema"] >= TILT_RED_DEG:
            st["red_run"] += 1
            st["yellow_run"] = 0
        elif st["angle_ema"] >= TILT_YELLOW_DEG:
            st["yellow_run"] += 1
            st["red_run"] = 0
        else:
            st["red_run"] = 0
            st["yellow_run"] = 0

        # 5) 이벤트 트리거 (낙상: lie 전이 + 속도)
        if (not was_lie) and is_lie:
            if vy_peak >= self.VY_FALL_THRESH and (det.t - st["last_event_t"] > self.STICKY_SEC):
                score = (st["angle_ema"] - self.LIE_ANGLE) + 0.5*(vy_peak/self.VY_FALL_THRESH)
                out.append(Event(det.track_id, det.t, "fall", max(0.0, score), st["angle_ema"], vy_peak))
                st["last_event_t"] = det.t

        # 6) 이벤트 트리거 (추락)
        if st["inversion_run"] >= self.INV_MIN_FRAMES and vy_peak >= self.VY_FREEFALL_HIGH:
            if self._ground_near(det) and is_lie and (det.t - st["last_event_t"] > self.STICKY_SEC):
                score = 1.0*(vy_peak/self.VY_FREEFALL_HIGH) + 0.5*(st["angle_ema"]/90.0)
                out.append(Event(det.track_id, det.t, "fall_from_height", score, st["angle_ema"], vy_peak))
                st["last_event_t"] = det.t

        # === 색상 기준 이벤트 보수화(연속 프레임 요구) ===
        if st["angle_ema"] >= TILT_RED_DEG and st["red_run"] >= self.RED_MIN_FRAMES:
            if det.t - st["last_event_t"] > self.STICKY_SEC:
                out.append(Event(det.track_id, det.t, "fall", 1.0, st["angle_ema"], vy_peak))
                st["last_event_t"] = det.t
        elif st["angle_ema"] >= TILT_YELLOW_DEG and st["yellow_run"] >= self.YELLOW_MIN_FRAMES:
            if det.t - st["last_event_t"] > self.STICKY_SEC and vy_peak >= self.YELLOW_MIN_VY:
                out.append(Event(det.track_id, det.t, "fall_warning", 0.5, st["angle_ema"], vy_peak))
                st["last_event_t"] = det.t

        return out

# ======================
# 간단한 센트로이드 트래커
# ======================
class CentroidTracker:
    def __init__(self, max_dist: float = 80.0, max_age: int = 30):
        self.next_id = 1
        self.tracks: Dict[int, Dict] = {}  # id -> {"c":(x,y),"age":0}
        self.max_dist = max_dist
        self.max_age = max_age

    @staticmethod
    def _dist(a: Tuple[float,float], b: Tuple[float,float]) -> float:
        return math.hypot(a[0]-b[0], a[1]-b[1])

    def update(self, centers: List[Tuple[float,float]]) -> List[int]:
        # 증가 age
        for tid in list(self.tracks.keys()):
            self.tracks[tid]["age"] += 1
            if self.tracks[tid]["age"] > self.max_age:
                del self.tracks[tid]

        assigned = set()
        ids_out: List[int] = [-1]*len(centers)

        # 1) 기존 트랙에 매칭
        for i, c in enumerate(centers):
            best_id, best_d = None, 1e9
            for tid, st in self.tracks.items():
                d = self._dist(c, st["c"])
                if d < best_d and d <= self.max_dist and tid not in assigned:
                    best_d, best_id = d, tid
            if best_id is not None:
                self.tracks[best_id]["c"] = c
                self.tracks[best_id]["age"] = 0
                ids_out[i] = best_id
                assigned.add(best_id)

        # 2) 새 트랙 생성
        for i, c in enumerate(centers):
            if ids_out[i] == -1:
                tid = self.next_id
                self.next_id += 1
                self.tracks[tid] = {"c": c, "age": 0}
                ids_out[i] = tid

        return ids_out

# ======================
# YOLO Pose 래퍼
# ======================
# ===== [UPDATED] PoseProviderYOLO: 극단자세 보강 + NMS 제어 버전 =====
class PoseProviderYOLO:
    """
    ultralytics YOLO Pose를 사용해 COCO 17 keypoints 기반 좌표를 뽑아 PosePoints 생성
    - 극단자세 보강(기존 설명 생략)
    - ★ NMS 제어: max_det, iou를 predict에 전달하여 과도한 후보 폭주 억제
    """
    def __init__(
        self,
        weights: str,
        imgsz: int = 640,
        conf: float = 0.25,
        device: str = "",
        use_tta: bool = True,              # 수평 플립 TTA
        gamma: float = 0.0,                # 0이면 꺼짐 (예: 1.4~1.8)
        use_clahe: bool = False,           # 저조도 보정
        min_visible_kpts: int = 4,         # 보이는 키포인트 최소 개수
        kpt_conf_thresh: float = 0.15,     # 키포인트 confidence 임계
        max_det: int = 80,                 # ★ NMS 이후 최종 최대 박스 수
        iou: float = 0.5                   # ★ NMS IoU 임계
    ):
        try:
            from ultralytics import YOLO
        except Exception as e:
            raise SystemExit("[ERROR] ultralytics가 필요합니다: pip install ultralytics") from e
        self.model = YOLO(weights)
        self.imgsz = imgsz
        self.conf = conf
        self.device = device
        self.use_tta = use_tta
        self.gamma = gamma
        self.use_clahe = use_clahe
        self.min_visible_kpts = min_visible_kpts
        self.kpt_conf_thresh = kpt_conf_thresh
        self.max_det = max_det
        self.iou = iou

    @staticmethod
    def _mid(a: Tuple[float,float], b: Tuple[float,float]) -> Tuple[float,float]:
        return ((a[0]+b[0])*0.5, (a[1]+b[1])*0.5)

    # 간단 전처리(감마, CLAHE)
    def _preprocess(self, frame):
        import cv2
        import numpy as np
        out = frame
        if self.gamma and self.gamma > 0 and abs(self.gamma-1.0) > 1e-3:
            lut = np.array([(i/255.0) ** (1.0/self.gamma) * 255 for i in range(256)]).astype("uint8")
            out = cv2.LUT(out, lut)
        if self.use_clahe:
            lab = cv2.cvtColor(out, cv2.COLOR_BGR2LAB)
            l, a, b = cv2.split(lab)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            l2 = clahe.apply(l)
            lab2 = cv2.merge([l2,a,b])
            out = cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)
        return out

    def _run_once(self, frame):
        # augment=True 로 멀티스케일/플립 일부 포함되지만, 수평 TTA는 아래에서 직접 병합
        results = self.model.predict(
            source=frame,
            imgsz=self.imgsz,
            conf=self.conf,
            device=self.device,
            verbose=False,
            augment=False,
            agnostic_nms=True,
            max_det=self.max_det,    # ★ 추가
            iou=self.iou             # ★ 추가
        )
        return results[0] if results else None

    def infer(self, frame) -> List[Dict]:
        """
        return: list of dict:
            {"bbox":(x,y,w,h), "kpts_xy": np.ndarray(shape (17,2)), "kpts_conf": np.ndarray(17,) or None, "center":(cx,cy)}
        """
        import numpy as np
        import cv2

        base = self._preprocess(frame)

        # 원본
        r0 = self._run_once(base)
        dets0 = self._to_dets(r0)

        # 수평 뒤집기 TTA
        dets = dets0
        if self.use_tta:
            flip = cv2.flip(base, 1)
            r1 = self._run_once(flip)
            dets1 = self._to_dets(r1)
            W = base.shape[1]
            # 좌우 반전 좌표 복원
            for d in dets1:
                x, y, w, h = d["bbox"]
                x = W - (x + w)
                d["bbox"] = (x, y, w, h)
                cx, cy = d["center"]
                d["center"] = (W - cx, cy)
                if d["kpts_xy"] is not None:
                    kxy = d["kpts_xy"].copy()
                    kxy[:,0] = W - kxy[:,0]
                    d["kpts_xy"] = kxy
            dets = self._merge_iou(dets0, dets1, iou_th=0.6)

        # 최소 가시점 필터(완화)
        out = []
        for d in dets:
            kxy = d["kpts_xy"]
            kcf = d.get("kpts_conf")
            if kxy is None:
                continue
            vis = self._visible_mask(kxy, kcf)
            if vis.sum() >= self.min_visible_kpts:
                out.append(d)
        return out

    def _to_dets(self, r) -> List[Dict]:
        import numpy as np
        out: List[Dict] = []
        if r is None or r.boxes is None:
            return out
        boxes_xywh = r.boxes.xywh.cpu().numpy() if r.boxes is not None else np.empty((0,4))
        kpts_xy = r.keypoints.xy.cpu().numpy() if r.keypoints is not None else None

        # 키포인트 confidence가 있는 경우
        kpts_conf = None
        try:
            if r.keypoints is not None and hasattr(r.keypoints, "conf"):
                kpts_conf = r.keypoints.conf.cpu().numpy()
        except Exception:
            kpts_conf = None

        for i in range(len(boxes_xywh)):
            x,y,w,h = boxes_xywh[i]
            center = (float(x), float(y))
            kxy = kpts_xy[i] if kpts_xy is not None and i < len(kpts_xy) else None
            kcf = kpts_conf[i] if kpts_conf is not None and i < len(kpts_conf) else None
            out.append({
                "bbox": (float(x - w/2), float(y - h/2), float(w), float(h)),
                "kpts_xy": kxy, "kpts_conf": kcf, "center": center
            })
        return out

    @staticmethod
    def _merge_iou(a: List[Dict], b: List[Dict], iou_th=0.6) -> List[Dict]:
        def iou(bb1, bb2):
            x1,y1,w1,h1 = bb1; x2,y2,w2,h2 = bb2
            A = (w1*h1); B = (w2*h2)
            ix1,iy1 = max(x1,x2), max(y1,y2)
            ix2,iy2 = min(x1+w1, x2+w2), min(y1+h1, y2+h2)
            iw, ih = max(0, ix2-ix1), max(0, iy2-iy1)
            inter = iw*ih
            u = A + B - inter
            return inter/u if u>0 else 0.0

        out = a[:]
        for db in b:
            if not any(iou(db["bbox"], da["bbox"]) > iou_th for da in a):
                out.append(db)
        return out

    def _visible_mask(self, kxy, kcf) -> "np.ndarray[bool]":
        import numpy as np
        if kxy is None:
            return np.zeros((0,), dtype=bool)
        if kcf is None:
            # conf가 없으면 좌표 유효성으로 판단
            return (kxy[:,0] > 0) & (kxy[:,1] > 0)
        return (kcf >= self.kpt_conf_thresh)

    def to_posepoints(self, det: Dict) -> Optional[PosePoints]:
        """
        COCO 17 keypoints index:
        0 nose, 5 L-shoulder, 6 R-shoulder, 11 L-hip, 12 R-hip, 13 L-knee, 14 R-knee, 15 L-ankle, 16 R-ankle
        - 발목 대체 규칙: ankles -> (knees -> hips)
        - 머리/상부 보강: nose 없으면 shoulders mid, 둘 다 없으면 (hips mid + shoulders mid) 보간
        """
        import numpy as np
        kxy = det.get("kpts_xy")
        kcf = det.get("kpts_conf")
        if kxy is None or (isinstance(kxy, np.ndarray) and kxy.shape[0] < 17):
            return None

        def ok(idx):
            if kcf is not None:
                return (kcf[idx] is not None) and (kcf[idx] >= self.kpt_conf_thresh)
            return (kxy[idx,0] > 0 and kxy[idx,1] > 0)

        def kp(idx, fallback=None):
            if ok(idx):
                return (float(kxy[idx,0]), float(kxy[idx,1]))
            return fallback

        nose = kp(0)
        l_sh = kp(5); r_sh = kp(6)
        l_hip = kp(11); r_hip = kp(12)
        l_kn  = kp(13); r_kn  = kp(14)
        l_an  = kp(15); r_an  = kp(16)

        # shoulders mid
        shoulders_mid = None
        if l_sh and r_sh:
            shoulders_mid = self._mid(l_sh, r_sh)

        # lower 우선순위: ankles → knees → hips
        lower = None
        if l_an and r_an:
            lower = self._mid(l_an, r_an)
        elif l_kn and r_kn:
            lower = self._mid(l_kn, r_kn)
        elif l_hip and r_hip:
            lower = self._mid(l_hip, r_hip)

        # head(상부) 우선순위: nose → shoulders_mid → 사용 가능한 조합
        head = nose if nose else (shoulders_mid if shoulders_mid else None)

        # upper: 기본은 shoulders_mid와 nose의 평균, 없으면 head 사용
        if head and shoulders_mid:
            upper = self._mid(head, shoulders_mid)
        elif head:
            upper = head
        elif shoulders_mid:
            upper = shoulders_mid
        else:
            # 최후: 힙이 있으면 힙을 upper로(극단자세 최소복원)
            if l_hip and r_hip:
                upper = self._mid(l_hip, r_hip)
            else:
                return None

        if lower is None:
            # 그래도 없으면 bbox로 근사(최소 보존)
            x,y,w,h = det["bbox"]
            lower = (x + w*0.5, y + h*0.95)

        if head is None:
            head = upper

        return PosePoints(upper=upper, lower=lower, head=head, ankles=lower)

def resolve_pose_weights_auto(path_or_name: str) -> str:
    """
    - 로컬 경로가 존재하면 그대로 사용
    - 파일명(yolo11*/yolo12*)이면 HF에서 먼저 다운로드 시도
    - HF 실패 시 그대로 파일명을 반환(ultralytics가 처리 시도)
    - 마지막으로 12가 실패하면 11로 폴백
    """
    import os
    if os.path.isfile(path_or_name):
        return path_or_name

    name = os.path.basename(path_or_name)
    try:
        from huggingface_hub import hf_hub_download
        if name.startswith("yolo12"):
            try:
                return hf_hub_download("ultralytics/yolo12", name)
            except Exception:
                # 12 실패 시 11로 폴백 시도
                alt = name.replace("yolo12", "yolo11")
                return hf_hub_download("ultralytics/yolo11", alt)
        elif name.startswith("yolo11"):
            return hf_hub_download("ultralytics/yolo11", name)
    except Exception:
        pass
    return name

# ======================
# 러너: 비디오 입력/저장
# ======================
def run_video(
    source: str,
    save_csv: Optional[str],
    save_video: Optional[str],
    pose_weights: str,
    imgsz: int = 640,
    conf: float = 0.25,
    device: str = "",
    fps_override: Optional[float] = None,
):
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise SystemExit(f"[ERROR] 비디오를 열 수 없습니다: {source}")

    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps_in = cap.get(cv2.CAP_PROP_FPS) or 30.0
    fps = fps_override or fps_in or 30.0

    # 저장 비디오
    vw = None
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        os.makedirs(os.path.dirname(save_video) or ".", exist_ok=True)
        vw = cv2.VideoWriter(save_video, fourcc, fps, (W, H))

    # CSV
    csv_fp, csv_wr = None, None
    if save_csv:
        os.makedirs(os.path.dirname(save_csv) or ".", exist_ok=True)
        csv_fp = open(save_csv, "w", newline="", encoding="utf-8")
        csv_wr = csv.writer(csv_fp)
        csv_wr.writerow(["frame", "time_sec", "track_id", "event", "score", "angle", "vy_peak"])

    # 프로바이더/트래커/디텍터
    pose = PoseProviderYOLO(
        pose_weights,
        imgsz=imgsz,
        conf=conf,
        device=device,
        use_tta=True,
        gamma=1.6,
        use_clahe=True,
        min_visible_kpts=4,
        kpt_conf_thresh=0.12,
        max_det=80,
        iou=0.5,
    )

    tracker = CentroidTracker(max_dist=80.0, max_age=30)
    fall = AngleFallDetector(fps=fps)

    # 사라짐 기반 추락 감지 파라미터
    DISAPPEAR_MISSED_FRAMES = 5
    DISAPPEAR_Y2_RATIO = 0.85
    DISAPPEAR_VY_THRESH = 800.0

    track_meta: Dict[int, Dict[str, float]] = {}

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        t_video = frame_idx / fps

        dets = pose.infer(frame)
        centers = [d["center"] for d in dets]
        tids = tracker.update(centers)

        present_ids = set()

        for d, tid in zip(dets, tids):
            present_ids.add(tid)
            pp = pose.to_posepoints(d)
            bbox = d["bbox"]
            det_in = DetectionIn(
                track_id=tid, t=t_video, pose=pp, bbox=bbox, frame_w=W, frame_h=H
            )
            events = fall.step(det_in)

            x,y,w,h = bbox
            x1,y1,x2,y2 = int(x), int(y), int(x+w), int(y+h)

            if pp:
                ux,uy = map(int, pp.upper)
                lx,ly = map(int, pp.lower)
                cv2.line(frame, (lx,ly), (ux,uy), (255,255,0), 2)
                axis_x = pp.upper[0] - pp.lower[0]
                axis_y = pp.upper[1] - pp.lower[1]
                theta = angle_to_ground_from_axis(axis_x, axis_y, vert=fall.VERT)
                draw_ground_angle_guides(frame, (lx, ly), theta_deg=theta, length=90)
            else:
                origin = (int(x + w*0.5), int(y + h))
                draw_ground_angle_guides(frame, origin, theta_deg=0.0, length=90)

            st = fall.state[tid]
            angle_ema = st["angle_ema"] if st["angle_ema"] is not None else 0.0
            if angle_ema >= TILT_RED_DEG:
                box_color = (0, 0, 255)
            elif angle_ema >= TILT_YELLOW_DEG:
                box_color = (0, 255, 255)
            else:
                box_color = (0, 255, 0)
            cv2.rectangle(frame, (x1,y1), (x2,y2), box_color, 2)

            win_t0 = t_video - fall.FALL_WINDOW_SEC
            vy_vals = [vy for (tt,vy) in st["vy_hist"] if tt >= win_t0]
            vy_peak = max(vy_vals) if vy_vals else 0.0
            label = f"id:{tid} tilt:{angle_ema:.1f} vy*:{vy_peak:.0f}"
            cv2.putText(frame, label, (x1, max(20,y1-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,255,200), 1, cv2.LINE_AA)

            for ev in events:
                if ev.kind == "fall":
                    cv2.putText(frame, "FALL!", (x1, y1-24), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2, cv2.LINE_AA)
                elif ev.kind == "fall_from_height":
                    cv2.putText(frame, "FALL FROM HEIGHT!", (x1, y1-24), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2, cv2.LINE_AA)
                elif ev.kind == "fall_warning":
                    cv2.putText(frame, "FALL WARNING", (x1, y1-24), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2, cv2.LINE_AA)
                if csv_wr:
                    csv_wr.writerow([frame_idx, f"{t_video:.3f}", ev.track_id, ev.kind, f"{ev.score:.3f}", f"{ev.angle:.2f}", f"{ev.vy_peak:.1f}"])

            tm = track_meta.get(tid, {"missed": 0})
            tm["last_y2"] = float(y2)
            tm["last_vy_peak"] = float(vy_peak)
            tm["last_angle"] = float(angle_ema)
            tm["last_t"] = float(t_video)
            tm["missed"] = 0
            track_meta[tid] = tm

        for tid in list(fall.state.keys()):
            if tid in present_ids:
                continue
            tm = track_meta.get(tid)
            if tm is None:
                continue
            tm["missed"] = tm.get("missed", 0) + 1
            track_meta[tid] = tm

            if tm["missed"] == DISAPPEAR_MISSED_FRAMES:
                y2_last = tm.get("last_y2", 0.0)
                vy_last = tm.get("last_vy_peak", 0.0)
                if (y2_last >= DISAPPEAR_Y2_RATIO * H) and (vy_last >= DISAPPEAR_VY_THRESH):
                    if csv_wr:
                        csv_wr.writerow([frame_idx, f"{t_video:.3f}", tid, "fall_from_height_exit",
                                         f"{vy_last/ max(1.0, DISAPPEAR_VY_THRESH):.3f}",
                                         f"{fall.state[tid]['angle_ema'] if fall.state[tid]['angle_ema'] else 0.0:.2f}",
                                         f"{vy_last:.1f}"])

        if vw:
            vw.write(frame)

    cap.release()
    if vw:
        vw.release()
    if csv_fp:
        csv_fp.close()

def parse_args():
    p = argparse.ArgumentParser(description="Angle-based Fall / Fall-from-height detector (YOLO Pose)")
    p.add_argument("--source", type=str, required=True, help="비디오 경로 또는 카메라 인덱스(예: 0)")
    p.add_argument("--pose-weights", type=str, required=True, help="YOLO pose 가중치 경로 (예: yolo11n-pose.pt)")
    p.add_argument("--imgsz", type=int, default=640)
    p.add_argument("--conf", type=float, default=0.25)
    p.add_argument("--device", type=str, default="", help="cuda:0 / cpu 등")
    p.add_argument("--fps", type=float, default=0.0, help="입력 FPS를 덮어쓸 값(0이면 원본 FPS 사용)")
    p.add_argument("--save-csv", type=str, default="", help="이벤트 CSV 저장 경로 (예: c:/out/events.csv)")
    p.add_argument("--save-video", type=str, default="", help="주석 영상 저장 경로 (예: c:/out/out.mp4)")
    return p.parse_args()

if __name__ == "__main__":
    # 직접 경로 지정(예시)
    source = "./video/fall_5.mp4"
    pose_weights = resolve_pose_weights_auto("yolo12s-pose.pt")
    save_video = "./output_video/fall_10.mp4"
    save_csv   = "./out/events.csv"

    run_video(
        source=source,
        save_csv=save_csv,
        save_video=save_video,
        pose_weights=pose_weights,
        imgsz=640,
        conf=0.25,
        device="cuda:0",
        fps_override=None
    )
